In [58]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

from sympy.physics.quantum.constants import hbar
from sympy import pi, symbols, integrate, oo, Sum, simplify, sinh